In [ ]:
import scqubits as scq
import numpy as np

fluxonium = scq.Fluxonium(
    EJ=3.395,
    EC=0.479,
    EL=0.132,
    flux=0.5,
    cutoff=50,
)

w_r = 5.7
Nr = 50
resonator = scq.Oscillator(
    E_osc=w_r,
    truncated_dim=Nr,
)

hilbertspace = scq.HilbertSpace([fluxonium, resonator])

g = 0.1
n_op = fluxonium.n_operator()
x_op = resonator.annihilation_operator() + resonator.creation_operator()

hilbertspace.add_interaction(
    g=g,
    op1=fluxonium.n_operator,
    op2=resonator.creation_operator,
    add_hc=True,
)


In [ ]:
N = 100
evals, evecs = hilbertspace.eigensys(evals_count=N)
hilbertspace.generate_lookup()

$$
P_{res} (n) = (1 - exp(-\beta \hbar \omega_r)) exp(-n \beta \hbar \omega_r) \\
n_{th}(\omega_j) = \frac{1}{exp(\beta \hbar \omega_j) - 1} \\
$$

In [ ]:
beta_hbar = 1
kappa = 0.1


def P_res(n):
    return (1 - np.exp(-beta_hbar * w_r)) * np.exp(-n * beta_hbar * w_r)


def n_th(w_j):
    return 1 / (np.exp(beta_hbar * w_j) - 1)

In [ ]:
Nmax = Nr // 10
ns = np.arange(0, Nmax)
g_idxs = np.array([hilbertspace.dressed_index((0, n)) for n in ns])
e_idxs = np.array([hilbertspace.dressed_index((1, n)) for n in ns])
Egs, Ees = evals[g_idxs], evals[e_idxs]  # (ns,)
Vgs, Ves = evecs[g_idxs], evecs[e_idxs]  # (ns, N)

In [ ]:
P_res_ns = P_res(ns)
E_1n0n = Egs[:, None] - Ees[None, :]  # (ns, ns), from |1, n> to |0, n'>

$$
\Gamma^{up}_{l->l'} = \sum_{n,n'} P_{res}(n)\kappa n_{th} (\omega_{l',n'} - \omega_{l,n})\left|\langle l',n'\left|a^\dagger\right|l,n \rangle\right|^2 \\
$$

In [ ]:
# calculate the transition rate of 0-1 caused by percell effect
up_mask = E_1n0n > 0
E_1n0n_up = E_1n0n.copy()
E_1n0n_up[~up_mask] = np.inf
n_ths = n_th(E_1n0n_up)  # (ns, ns)
# calculate <0, n'|a|1, n>
a_op = scq.identity_wrap(
    resonator.annihilation_operator(), resonator, [fluxonium, resonator]
)
a_1n0n = np.zeros((Nmax, Nmax), dtype=complex)
for ng in ns:
    for ne in ns:
        a_1n0n[ng, ne] = Vgs[ng].dag() * a_op * Ves[ne]
Percell_up = np.sum(P_res_ns[:, None] * kappa * n_ths * np.abs(a_1n0n) ** 2).item()
Percell_up

$$
\Gamma^{down}_{l->l'} = \sum_{n,n'} P_{res}(n)\kappa (n_{th} (-\omega_{l',n'} + \omega_{l,n}) + 1)\left|\langle l',n'\left|a\right|l,n \rangle\right|^2 \\
$$

In [ ]:
# calculate the transition rate of 0-1 caused by percell effect
down_mask = E_1n0n < 0
E_1n0n_down = -E_1n0n.copy()
E_1n0n_down[~down_mask] = np.inf
n_ths = n_th(E_1n0n_down)  # (ns, ns)
# calculate <0, n'|a|1, n>
ad_op = scq.identity_wrap(
    resonator.creation_operator(), resonator, [fluxonium, resonator]
)
ad_1n0n = np.zeros((Nmax, Nmax), dtype=complex)
for ng in ns:
    for ne in ns:
        ad_1n0n[ng, ne] = Vgs[ng].dag() * ad_op * Ves[ne]
Percell_down = np.sum(P_res_ns[:, None] * kappa * n_ths * np.abs(ad_1n0n) ** 2).item()
Percell_down

# Sweep flux

In [ ]:
import scqubits as scq
import numpy as np

EJ = 3.395
EC = 0.479
EL = 0.132
g = 0.1
w_r = 5.7
N = 100
Nr = 100
T = 60e-3  # K
k = 1.380649e-23
hbar = 1.0545718e-34
beta_hbar = hbar / (k * T) * 1e9
kappa = 0.001


def P_res(n):
    return (1 - np.exp(-beta_hbar * w_r)) * np.exp(-n * beta_hbar * w_r)


def n_th(w_j):
    return 1 / (np.exp(beta_hbar * w_j) - 1)


fluxonium = scq.Fluxonium(
    EJ=EJ,
    EC=EC,
    EL=EL,
    flux=0.0,
    cutoff=N,
)

resonator = scq.Oscillator(
    E_osc=w_r,
    truncated_dim=Nr,
)

hilbertspace = scq.HilbertSpace([fluxonium, resonator])

hilbertspace.add_interaction(
    g=g,
    op1=fluxonium.n_operator,
    op2=resonator.creation_operator,
    add_hc=True,
)


def percell(flx, evals, evecs):
    global N, Nr, beta_hbar, kappa, fluxonium, resonator, hilbertspace

    fluxonium.flux = flx
    # hilbertspace.generate_lookup()

    # calculate the transition rate of 0-1 caused by percell effect
    Nmax = N // 6  # truncate the resonator space
    ns = np.arange(0, Nmax)
    g_idxs = np.array([hilbertspace.dressed_index((0, n)) for n in ns])
    e_idxs = np.array([hilbertspace.dressed_index((1, n)) for n in ns])
    Egs, Ees = evals[g_idxs], evals[e_idxs]  # (ns,)
    Vgs, Ves = evecs[g_idxs], evecs[e_idxs]  # (ns, N)

    P_res_ns = P_res(ns)
    E_1n0n = Egs[:, None] - Ees[None, :]  # (ns, ns), from |1, n> to |0, n>

    # calculate the transition rate of 0-1 caused by up percell effect
    ad_op = scq.identity_wrap(
        resonator.creation_operator(), resonator, [fluxonium, resonator]
    )
    up_mask = E_1n0n > 0
    E_1n0n_up = E_1n0n.copy()
    E_1n0n_up[~up_mask] = np.inf
    n_ths = n_th(E_1n0n_up)  # (ns, ns)
    # calculate <0, n'|a^dag|1, n>
    ad_1n0n = np.zeros((Nmax, Nmax), dtype=complex)
    for ng in ns:
        for ne in ns:
            ad_1n0n[ng, ne] = Vgs[ng].dag() * ad_op * Ves[ne]

    Percell_up = np.sum(P_res_ns[:, None] * kappa * n_ths * np.abs(ad_1n0n) ** 2).item()

    # calculate the transition rate of 0-1 caused by down percell effect
    a_op = scq.identity_wrap(
        resonator.annihilation_operator(), resonator, [fluxonium, resonator]
    )
    down_mask = E_1n0n < 0
    E_1n0n_down = -E_1n0n.copy()
    E_1n0n_down[~down_mask] = np.inf
    n_ths = n_th(E_1n0n_down)  # (ns, ns)
    # calculate <0, n'|a|1, n>
    a_1n0n = np.zeros((Nmax, Nmax), dtype=complex)
    for ng in ns:
        for ne in ns:
            a_1n0n[ng, ne] = Vgs[ng].dag() * a_op * Ves[ne]

    Percell_down = np.sum(
        P_res_ns[:, None] * kappa * n_ths * np.abs(a_1n0n) ** 2
    ).item()

    return Percell_up + Percell_down

In [ ]:
flxs = np.linspace(-0.4, 0.6, 101)


def update_hilbertspace(flx):
    fluxonium.flux = flx


sweep = scq.ParameterSweep(
    hilbertspace=hilbertspace,
    paramvals_by_name={"flux": flxs},
    update_hilbertspace=update_hilbertspace,
    evals_count=N,
    subsys_update_info={"flux": [fluxonium]},
)

In [ ]:
purcells = []
hilbertspace.generate_lookup()
for flx, evals, evecs in zip(flxs, sweep["evals"], sweep["evecs"]):
    purcells.append(percell(flx, evals, evecs))
purcells = np.array(purcells)

In [ ]:
import matplotlib.pyplot as plt

T1s = 1.0 / purcells * 1e-6
plt.plot(flxs, T1s)
# plt.xlim(0.4, 0.6)
plt.xlabel("flux")
plt.ylabel("T1 (ms)")
plt.show()